In [17]:
# import all libraries 
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image
import os
import uuid
import json



In [18]:
#read xml and get important items

def read_content(xml_file: str):

    tree = ET.parse(xml_file)
    root = tree.getroot()

    list_with_all_boxes = []
    list_with_all_names = []

    for boxes in root.iter('object'):

        file_name = root.find('filename').text
        file_path = root.find('path').text
        name = boxes.find('name').text

        ymin, xmin, ymax, xmax = None, None, None, None

        for box in boxes.findall("bndbox"):
            ymin = int(box.find("ymin").text)
            xmin = int(box.find("xmin").text)
            ymax = int(box.find("ymax").text)
            xmax = int(box.find("xmax").text)

        list_with_single_boxes = [xmin, ymin, xmax, ymax]
        list_with_all_boxes.append(list_with_single_boxes)
        list_with_all_names.append(name)

    return file_path,file_name, list_with_all_boxes,list_with_all_names

In [19]:
w=[]
for file in os.listdir("local/img"):
    if file.endswith(".xml"):
        w.append(read_content(file))
        #Image.open()
        

FileNotFoundError: [Errno 2] No such file or directory: 'IMG_20200921_142309.xml'

In [7]:
w

[]

In [ ]:
#load dictionary
with open('labels_map.json') as f:
    dic = json.loads(f.read())

In [ ]:
#crea recosres y metadata

if os.path.exists("recortes")==True:
    metadata=pd.read_csv("metadata.csv")
else:
    with open('labels_map.json') as f:
        dic = json.loads(f.read())

    os.mkdir('recortes')
    names,splits,labl=[],[],[]
    for i in w:
        p=Image.open('origin/'+i[1])
        p = np.asarray(p)

        for j in range(len(i[3])):
            a=p[i[2][j][1]:i[2][j][3],i[2][j][0]:i[2][j][2],0:3]
            a=Image.fromarray(a)
            name=uuid.uuid4().hex
            names.append(name+'.png')
            a.save('recortes/'+name+'.png')
            split = np.random.rand(1)[0]
            if split<0.7:
                spl='train'
            else:
                spl='test'
            splits.append(spl)
            labl.append(i[3][j])

    df={}
    df['names']=names
    df['split']=splits
    df['label']=labl
    df = pd.DataFrame(df)
    df=df.replace(dic)
    df.to_csv('metadata.csv')
    metadata=df
    


In [ ]:
def build_sources_from_metadata(metadata, data_dir, mode='train', exclude_labels=None):
    
    if exclude_labels is None:
        exclude_labels = set()
    if isinstance(exclude_labels, (list, tuple)):
        exclude_labels = set(exclude_labels)

    df = metadata.copy()
    df = df[df['split'] == mode]
    df['filepath'] = df['names'].apply(lambda x: os.path.join(data_dir, x))
    include_mask = df['label'].apply(lambda x: x not in exclude_labels)
    df = df[include_mask]

    sources = list(zip(df['filepath'], df['label']))
    return sources

In [ ]:
train_sources = build_sources_from_metadata(metadata, 'recortes')
test_sources = build_sources_from_metadata(metadata, 'recortes',mode='test')

In [ ]:
train_sources